In [55]:
import psycopg2
import numpy as np
from os import environ
conn = psycopg2.connect(dbname='postgres', user='postgres', host=environ['REDHAT_POSTGRES_1_PORT_5432_TCP_ADDR'])
cur = conn.cursor()

In [2]:
cur.execute("SELECT COUNT(*) FROM people"); print(cur.fetchone())
cur.execute("SELECT COUNT(*) FROM action"); print(cur.fetchone())
cur.execute("SELECT COUNT(*) FROM one_hot_ppl_act"); print(cur.fetchone())

(189118,)
(2695978,)
(7287,)


In [3]:
cur.execute("""SELECT column_name, data_type, character_maximum_length 
             FROM INFORMATION_SCHEMA.COLUMNS where table_name='one_hot_ppl_act'""")
columns = cur.fetchall()

In [5]:
keys = ', '.join([key[0] for key in columns[6:-1]]+['act_outcome'])

In [56]:
cur.execute("SELECT ({}) FROM one_hot_ppl_act WHERE (act_outcome = true OR act_outcome = false)".format(keys))
cur.fetchone()

('(f,t,f,t,f,f,f,f,f,f,f,f,f,f,f,f,f,f,f,f,f,f,f,f,f,f,f,f,f,f,f,f,f,f,f,f,f,f,f,f,f,f,f,t,f,f,f,f,f,f,f,f,f,f,f,f,f,f,f,f,f,f,f,f,f,f,f,f,t,f,f,f,f,f,f,f,f,t,f,f,f,f,f,f,t,f,f,f,f,f,f,t,f,f,f,f,f,f,f,f,f,f,f,f,f,f,f,f,f,f,f,f,f,f,f,t,f,f,f,f,f,f,f,t,f,f,f,f,f,f,f,t,f,f,t,t,f,t,f,f,f,f,t,f,f,f,f,f,t,t,f,t,t,f,f,t,t,t,f,36,f,f,f,t,f,f,f,t,f,f,f,f,f,f,f,f,f,f,f,f,f,f,f,f,f,f,f,f,f,f,f,f,f,f,f,f,f,f,f,f,f,f,f,f,f,f,f,f,f,f,f,f,f,f,f,f,f,f,f,t,f,f,f,f,f,f,f,f,f,f,f,f,f,f,f,f,f,f,f,f,f,f,f,f,f,f,f,f,f,f,f,f,t,f,f,f,f,f,f,f,f,f,f,f,t,f,f,f,f,f,f,f,t,f,f,f,f,f,f,f,t,f,f,f,f,f,t,f,f,f,f,f,f,f,f,t,f,f,f,f,f,f,f,f,f,f,f,f,f,f,f,f,f,f,t,f,f,f,f,f,f,f,f,f,f,f,f,f,f,f,f,f,f,f,f)',)

In [43]:

def one_hot_from_table(cursor):
    one_hot_vec = (cursor.fetchone()[0]
     .replace('t','1')
     .replace('f','0')
     .replace('(','')
     .replace(')','')
     .split(',')
    )
    one_hot_vec = [float(el) for el in one_hot_vec]
    outcome = one_hot_vec[-1:][0]
    one_hot_vec = one_hot_vec[:-1]
    return np.array(one_hot_vec), outcome

In [51]:
one_hot_from_table(cur)

(array([  0.,   1.,   0.,   0.,   1.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   1.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   1.,   0.,   0.,   0.,   0.,
          1.,   0.,   0.,   0.,   0.,   0.,   0.,   1.,   0.,   0.,   0.,
          0.,   0.,   0.,   1.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   1.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   1.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   1.,   1.,   0.,   0.,   0.,   1.,   0.,   0.,   0.,   0.,
          0.,   1.,   0.,   1.,   1., 

In [32]:
this_one_hot_vec, this_outcome = one_hot_from_table(cur)

In [8]:
import os; os.chdir('../')
from numpy.random import random
from lib.nn.perceptron import Perceptron

In [33]:
p = Perceptron(random(len(this_one_hot_vec)+1))

In [34]:
p.activate(this_one_hot_vec)

array(1.0)

In [35]:
this_outcome

0.0

In [65]:
def one_hot_from_table(cursor):
    one_hot_vec = (cursor.fetchone()[0]
     .replace('t','1')
     .replace('f','0')
     .replace('(','')
     .replace(')','')
     .split(',')
    )
    one_hot_vec = [float(el) for el in one_hot_vec]
    outcome = one_hot_vec[-1:][0]
    one_hot_vec = one_hot_vec[:-1]
    return np.array(one_hot_vec), outcome

this_one_hot_vec, this_outcome = one_hot_from_table(cur)

p = Perceptron(random(len(this_one_hot_vec)+1))

In [66]:
%%timeit


cur.execute("SELECT ({}) FROM one_hot_ppl_act WHERE (act_outcome = true OR act_outcome = false)".format(keys))

for _ in range(7000):
    this_one_hot_vec, this_outcome = one_hot_from_table(cur)
    p.update(this_one_hot_vec,this_outcome)

1 loop, best of 3: 2.08 s per loop


In [67]:
this_one_hot_vec, this_outcome = one_hot_from_table(cur)
p.activate(this_one_hot_vec), this_outcome

(array(0.0), 0.0)